In [ ]:
using Pkg
Pkg.activate("../../environments/refined-delaunay-for-flow-problems/");

In [ ]:
using LinearAlgebra
using Random
using SparseArrays
using Statistics

using AbstractPlotting
using CairoMakie
using Distances
using JuMP
using LightGraphs
using Parameters
using SCIP
using TriangleMesh

# Random Instance Data

In [ ]:
# create instance data
Random.seed!(0);

const N = 7
const WIDTH = 500
const HEIGHT = 300

x = round.(0.95 * WIDTH * rand(N))
y = round.(0.95 * HEIGHT * rand(N))
points = [x y]

# Triangulation with TriangleMesh

In [ ]:
"Create polygon from points and edges (used as segments)."
function Polygon(points, edges)
    n_point = size(points, 1)
    n_point_marker = 0
    n_point_attribute = 0
    n_segment = size(edges, 1)
    n_hole = 0
    poly = Polygon_pslg(n_point, n_point_marker, n_point_attribute, n_segment, n_hole)
    set_polygon_point!(poly, points)
    set_polygon_segment!(poly, edges)
    return poly
end

In [ ]:
struct TriangleSwitches
end

TriangleSwitches

# Drawing with Makie

In [ ]:
function make_scene(width=WIDTH, height=HEIGHT)
    return Scene(resolution=(width, height), show_axis=false, scale_plot=false)
end

In [ ]:
function draw_points!(points; markersize=4, color=:black)
    scatter!(points[:, 1], points[:, 2], markersize=markersize, color=color)
end

In [ ]:
make_scene()
draw_points!(points)

In [ ]:
function draw_edges!(points, edges; color=:gray)
    linesegments!(points[edges'[:], :], color=color)
end

In [ ]:
function draw_triangulation(points, edges)
    make_scene()
    draw_edges!(points, edges)
    draw_points!(points)   
end

In [ ]:
draw_triangulation(points, [1 2; 2 3; 3 4; 4 5; 5 6; 6 7; 7 1])

# Triangle Centers

In [ ]:
pointset_mean(array) = dropdims(mean(array, dims=2), dims=2)

In [ ]:
abstract type TriangleCenter end

struct TriangleCentroid <: TriangleCenter end
struct TriangleIncenter <: TriangleCenter end
struct TriangleCircumcenter <: TriangleCenter end # yield Voronoi points!

In [ ]:
function triangle_centers(triangulation, ::TriangleCentroid)
    @unpack points, triangles = triangulation
    return pointset_mean(points[triangles, :])
end

triangle_centers(t) = triangle_centers(t, TriangleCentroid())

In [ ]:
function triangle_centers(triangulation, ::TriangleIncenter)
    @unpack points, triangles = triangulation
    centers = []
    for t in eachrow(triangles)
        corners = points[t, :]

        a = norm(corners[2, :] - corners[3, :])
        b = norm(corners[1, :] - corners[3, :])
        c = norm(corners[1, :] - corners[2, :])
        # based on barycentric coordinates a:b:c        
        incenter = [a b c] * corners ./ (a + b + c)
        push!(centers, incenter)
    end
    
    return vcat(centers...)
end

In [ ]:
function triangle_centers(triangulation, ::TriangleCircumcenter)
    @unpack points, triangles = triangulation
    centers = []
    for t in eachrow(triangles)
        corners = points[t, :]
        Ax, Ay = corners[1, :]
        Bx, By = corners[2, :]
        Cx, Cy = corners[3, :]
        D = 2 * ( Ax * (By - Cy) + Bx * (Cy - Ay) + Cx * (Ay - By) )
        Ux = ((Ax^2 + Ay^2)*(By - Cy)) + ((Bx^2 + By^2)*(Cy - Ay)) + ((Cx^2 + Cy^2)*(Ay - By))
        Uy = ((Ax^2 + Ay^2)*(Cx - Bx)) + ((Bx^2 + By^2)*(Ax - Cx)) + ((Cx^2 + Cy^2)*(Bx - Ax))
        circumcenter = [Ux Uy] ./ D
        push!(centers, circumcenter)
    end
    
    return vcat(centers...)
end

In [ ]:
draw_triangulation(del)
draw_points!(triangle_centers(del, TriangleCentroid()), color=:limegreen)
draw_points!(triangle_centers(del, TriangleIncenter()), color=:magenta)
draw_points!(triangle_centers(del, TriangleCircumcenter()), color=:lightblue)

# Edge Subdivision

In [ ]:
function edge_midpoints(triangulation)
    @unpack points, edges = triangulation
    return pointset_mean(points[edges, :])
end

In [ ]:
function subdivided_edges(edges, offset)
    set = Vector()
    for e in 1:size(edges, 1)
        edge = edges[e, :]
        push!(set, [edge[1] e + offset])
        push!(set, [e + offset edge[2]])
    end
    return vcat(set...)
end

# Steiner Tree Model

In [ ]:
function antiparallel_digraph(triangulation)
    @unpack points, edges = triangulation
    graph = SimpleDiGraph(size(points, 1))
    for e in 1:size(edges, 1)
        s, t = edges[e, :]
        add_edge!(graph, s, t)
        add_edge!(graph, t, s)
    end
    return graph
end

In [ ]:
function edge_lengths(points, edges)
    diff = points[edges[:, 1], :] - points[edges[:, 2], :]
    return dropdims(mapslices(norm, diff, dims=2), dims=2)
end

edge_lengths(triangulation) = edge_lengths(triangulation.points, triangulation.edges)

In [ ]:
function edge_length_map(triangulation)
    @unpack edges = triangulation
    lengths = edge_lengths(triangulation)
    
    length_map = Dict{Tuple{Int64, Int64}, Float64}()
    for e = 1:size(edges, 1)
        s, t = edges[e, :]
        length_map[s, t] = lengths[e]
        length_map[t, s] = lengths[e]
    end
    return length_map
end

In [ ]:
function steiner_tree(triangulation, terminals)
    # Compute length by Euclidean distance of nodes.
    lengths = edge_length_map(triangulation)
    
    # Build digraph with all antiparallel arcs, for nonnegative flow.
    graph = antiparallel_digraph(triangulation)
    nodes = collect(1:nv(graph))
    arcs = collect(keys(lengths))
    
    length(terminals) >= 2 || error("Need at least 2 terminals.")
    all(terminals .<= nv(graph)) || error("Terminals out of range.")
    root = terminals[1]
    sinks = terminals[2:end]
   
    demand(v, s) = 1.0*(v == s) - 1.0*(v == root)
    
    # Using arc length for fixed capacity cost and multi-commodity flow.
    model = JuMP.direct_model(SCIP.Optimizer(display_verblevel=0))
    @variable(model, select[a in arcs], Bin, container=SparseAxisArray)
    @variable(model, flow[a in arcs,s in sinks] ≥ 0, container=SparseAxisArray)
    @constraint(model, balance[v in nodes, s in sinks],
        sum(flow[(n, v), s] - flow[(v, n), s] for n in neighbors(graph, v))
        == demand(v, s))
    @constraint(model, capacity[a in arcs, s in sinks], flow[a, s] <= select[a])
    @objective(model, Min, sum(lengths[a] * select[a] for a in arcs))
    
    optimize!(model)
    
    return objective_value(model), value.(select)

end

In [ ]:
function draw_tree(triangulation, terminals)
    @unpack points, edges = triangulation

    obj, select = steiner_tree(triangulation, terminals)
    @show obj
    selected = [select[(s,t)] + select[(t,s)] for (s,t) in eachrow(edges)]
    active_edges = edges[selected .> 0.0, :]
    
    draw_triangulation(triangulation)
    linesegments!(points[active_edges'[:], :], color=:plum, linewidth=3)
    draw_points!(points[terminals, :], markersize=5, color=:teal)
end

In [ ]:
terminals = collect(1:7);

# Distances and Shortest Paths

In [ ]:
function graph(triangulation)
    @unpack points, edges = triangulation
    graph = SimpleGraph(size(points, 1))
    for e in 1:size(edges, 1)
        s, t = edges[e, :]
        add_edge!(graph, s, t)
    end
    return graph
end

In [ ]:
function distance_matrix(triangulation)
    @unpack points = triangulation
    return pairwise(Euclidean(), points, points, dims=1)
end

In [ ]:
"Shortest path distance between two opposing vertices of adjacent triangles."
function shortest_path(graph, s, t, distances)
    heur(n) = distances[n, t]
    path = a_star(graph, s, t, distances, heur)
    return sum(distances[src(edge), dst(edge)] for edge in path)
end

# TODO: clean-up the stuff below

In [ ]:
# can do simple Delaunay triangulation
mesh = create_mesh(points, prevent_steiner_points=true, delaunay=true);
draw_triangulation(Triangulation(mesh))

In [ ]:
# refine the delaunay triangulation with defaults
mesh2 = create_mesh(polygon)
draw_tree(Triangulation(mesh2), 1:7)

In [ ]:
# limit number of Steiner points
mesh2 = create_mesh(polygon, "pQeDS1")
draw_tree(Triangulation(mesh2), 1:7)

In [ ]:
mesh2 = create_mesh(polygon, "pQea1000qS50")
draw_tree(Triangulation(mesh2), 1:7)

In [ ]:
# not keeping Delaunay edges (unconstrained)
mesh2 = create_mesh(points, "cQea1000qS50D")
draw_tree(Triangulation(mesh2), 1:7)

In [ ]:
# combine it with shortcut subdivision
polygon = Polygon(constrained_longest_subdivision2(del2))
mesh = create_mesh(polygon, "pQea2000S30")
draw_tree(Triangulation(mesh), 1:7)

Not bad (above), inserting some trianger centers, but also subdividing edges.

In [ ]:
# combine it with shortcut subdivision (make it conformant)
polygon = Polygon(constrained_longest_subdivision2(del2))
mesh = create_mesh(polygon, "pQea2000S30D")
draw_tree(Triangulation(mesh), 1:7)

Making it conformant (Delaunay) actually makes it worse.

Is it because for many "line-of-sight" directions, there are too many "crossing edges"?

In [ ]:
# combine it with shortcut subdivision (quality triangles with large angles)
polygon = Polygon(constrained_longest_subdivision2(del2))
mesh = create_mesh(polygon, "pQea2000S30q")
draw_tree(Triangulation(mesh), 1:7)

Imposing "quality triangles" is similarly bad.

In [ ]:
# combine it with shortcut subdivision (disable quality triangles with large angles)
polygon = Polygon(constrained_longest_subdivision2(del2))
mesh = create_mesh(polygon, "pQea2000S30q0")
draw_tree(Triangulation(mesh), 1:7)

In [ ]:
# try same setting without added edge subdivision as segment
polygon = Polygon(del2)
mesh = create_mesh(polygon, "pQea2000S30q0")
draw_tree(Triangulation(mesh), 1:7)

Maybe was just lucky with insertion of extra segment?

In [ ]:
# combine it with shortcut subdivision (quality triangles with large angles)
mesh = create_mesh(points, "cQea2000S30q0")
draw_tree(Triangulation(mesh), 1:7)